In [1]:
#!/usr/bin/env python3

import glob
import os
from os.path import join
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch 
from dataset_utils import MoNuSegDataset
from model import CNN3
from PIL import Image 
from eval_train import eval_model

In [2]:
train = True
ROOT = '/share/DEEPLEARNING/datasets/monuseg/'
TRAIN_IMG = join(ROOT,'MoNuSegTrainingData')
TEST_IMG = join(ROOT,'MoNuSegTestData')
if train:
    IMGS_DIR = join(TRAIN_IMG,'Tissue_Images')
    MASKS_DIR = join(TRAIN_IMG,'Binary_masks')
    OUTPUT_DIR = join(TRAIN_IMG,'Output')
else:
    IMGS_DIR = join(TEST_IMG,'Tissue_Images')
    MASKS_DIR = join(TEST_IMG,'Binary_masks')
    OUTPUT_DIR = join(TEST_IMG,'Output')


In [3]:
MODEL_DIR = '/share/homes/karmimy/equiv/save_model/fully_supervised_monuseg/11'
MODEL_NAME = 'fully_supervised_monuseg.pt'
gpu = 1

In [4]:
test_dataset = MoNuSegDataset(ROOT,image_set='test',load_entire_image=True)
dataloader_val = torch.utils.data.DataLoader(test_dataset,num_workers=4,pin_memory=True,\
            batch_size=1)
device =  torch.device("cuda:"+str(gpu) if torch.cuda.is_available() else "cpu")

In [5]:
model = torch.load(join(MODEL_DIR,MODEL_NAME))
model.to(device)

 )
    )
    (layer2): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): Bottleneck(
        (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(12

In [6]:
state = eval_model(model,dataloader_val,device=device,num_classes=4)

In [7]:
state

State:
	iteration: 14
	epoch: 1
	epoch_length: 14
	max_epochs: 1
	output: <class 'tuple'>
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: torch.utils.data.dataloader.DataLoader
	seed: <class 'NoneType'>
	times: <class 'dict'>

In [10]:
iou = state.metrics['IoU']
miou = state.metrics['mean IoU']
acc = state.metrics['accuracy']
loss = state.metrics['CE Loss'] 
dice_coeff = state.metrics['dice_coeff']
precision = state.metrics['precision']
recall = state.metrics['recall']

In [11]:
iou

tensor([0.8503, 0.5419, 0.0000], dtype=torch.float64)

In [12]:
miou

0.46403683349627833

In [13]:
dice_coeff.mean()

tensor(0.5406, dtype=torch.float64)